## 1. join 道路link基本数据和要素数据

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Test") \
    .config('spark.default.parallelism','2') \
    .config('spark.executor.cores','2') \
    .getOrCreate()

df_link = spark.read.format("com.databricks.spark.csv") \
.option("delimiter", "\t") \
.load("/Users/andyzhuang/git/maps_arch/road_sampling/data/link_grid_lenth_zoom15_version2.csv") \
.toDF("link_id", "adcode", "region", "grid_id", "length")

df_link.show(5)
df_link.createTempView("temp_link") 

df_elem = spark.read.format("com.databricks.spark.csv") \
.option("delimiter", " ") \
.load("/Users/andyzhuang/git/maps_arch/road_sampling/data/road_elem.csv") \
.toDF("link_id", 'lane','cond','light','eye','rev','adj')
# lane', 车信
# 'cond',交限
# 'light',信号灯
# 'eye',电子眼
# 'rev',掉头口
# 'adj',路口挂接

df_elem.show(5)
df_elem.createTempView("temp_elem") 


df_link_mid = spark.read.format("com.databricks.spark.csv") \
.option("delimiter", ",") \
.load("/Users/andyzhuang/Documents/0-业务/地图数据说明/yuangang/all_214wan.mid") \
.toDF("link_id", "kind", "length", "adcode", "unknown")

df_link_mid.show(5)
df_link_mid.createTempView("temp_link_mid") 




+-------------+------+------+---------------+------+
|      link_id|adcode|region|        grid_id|length|
+-------------+------+------+---------------+------+
|     13194124|610429|  西北|310231212030203|   364|
|9100051363998|610528|  西北|310231211231222|    15|
|9100042483150|610827|  西北|310233120101302|    11|
|9100055577627|610524|  西北|310231302301223|    47|
|9100042071344|610326|  西北|310231023233022|   224|
+-------------+------+------+---------------+------+
only showing top 5 rows

+-------------+----+----+-----+---+---+---+
|      link_id|lane|cond|light|eye|rev|adj|
+-------------+----+----+-----+---+---+---+
|9100023785557|   0|   0|    0|  0|  0|  0|
|       378465|   0|   0|    0|  0|  0|  0|
|     52313639|   0|   0|    0|  0|  0|  0|
|    102920609|   0|   0|    0|  3|  0|  0|
|     34883016|   0|   0|    0|  0|  0|  0|
+-------------+----+----+-----+---+---+---+
only showing top 5 rows

+-------------+----+-------------+------+-------+
|      link_id|kind|       length|adc

In [9]:
df_link_elem = spark.sql("select a.link_id, a.adcode, a.region, a.grid_id, kind, a.length, nvl(lane,0) lane, nvl(cond,0) cond, nvl(light,0) light, nvl(eye,0) eye, nvl(rev,0) rev, \
           nvl(adj,0) adj \
          from temp_link a \
          left join temp_elem b on a.link_id=b.link_id \
          left join temp_link_mid c on a.link_id=c.link_id")

output_path = "/Users/andyzhuang/git/maps_arch/road_sampling/data/link_all_elem_kind.csv"
df_link_elem.repartition(1).write.mode('overwrite').csv(output_path, sep='\t', quote='\u0000')

In [10]:
df_link_elem.show(5)

+--------+------+------+---------------+----+------+----+----+-----+---+---+---+
| link_id|adcode|region|        grid_id|kind|length|lane|cond|light|eye|rev|adj|
+--------+------+------+---------------+----+------+----+----+-----+---+---+---+
|10000723|530112|  西南|310210210202233|  06|    19|   0|   0|    0|  0|  0|  0|
| 1000665|620725|  西北|310223133332301|  00|    90|   0|   0|    0|  0|  0|  0|
| 1000795|441900|  华南|310300012032113|  04|    20|   0|   0|    0|  0|  0|  2|
| 1000839|441900|  华南|310300012032112|  04|   209|   0|   0|    0|  0|  0|  1|
|10009009|440114|  华南|310300020113213|  03|    24|   0|   0|    0|  0|  0|  0|
+--------+------+------+---------------+----+------+----+----+-----+---+---+---+
only showing top 5 rows

